<a href="https://colab.research.google.com/github/Dracomp89/Eduardo-Phillips---202115611/blob/main/Complementaria_09_EP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Complementaria métodos computacionales 1
>
> Semana 09

***

# 1) Descenso gradiente (4pt)

Dado un set de datos (dos arrays 1-d) `x` y `y`, y una función definida como `f(x,alpha)`, implemente una función que encuentre, mediante descenso gradiente, el vector de parámetros `alpha` que minimice la suma de cuadrados de los errores de `y` con la función de modelo `f(x,alpha)`. Es decir,

$$
\min_{\vec{\alpha}} \sum_{i=1}^N \left(y_i - f(x_i,\vec{\alpha})\right)^2
$$

La función debe tener la siguiente signatura:

```python
def gradient_descent_least_squares(model_function, x_data, y_data, initial_parameters, learning_rate):
```


Para probar su algoritmo, invéntese alguna función no lineal (por ejemplo $\alpha_1 e^{-\alpha_1 x} \sin(\alpha_2 x)$, o algo así), genere datos de prueba, y agrégueles ruido, e intente encontrar los parámetros con los cuales generó los datos.

También hay un ejemplo de datos en bloque neón. Intente encontrar la función a la cual corresponden esos datos.


In [11]:
import numpy as np

# Definir la función no lineal para el ajuste
def model_function(x, alpha):
    return alpha[0] * np.exp(-alpha[1] * x) * np.sin(alpha[2] * x)

# Función de descenso por gradiente
def gradient_descent_least_squares(model_function, x_data, y_data, initial_parameters, learning_rate, max_iterations=1000, tolerance=1e-6):
    alpha = np.array(initial_parameters, dtype=np.float64)
    N = len(y_data)

    for iteration in range(max_iterations):
        # Calcular los valores predichos por el modelo
        predictions = model_function(x_data, alpha)

        # Calcular el error entre las predicciones y los datos reales
        error = predictions - y_data

        # Calcular el gradiente
        grad = np.zeros_like(alpha)
        for i in range(len(alpha)):
            # Derivada numérica del modelo con respecto a cada parámetro
            alpha_step = np.copy(alpha)
            alpha_step[i] += 1e-5  # Pequeño incremento para la derivada numérica
            grad[i] = (np.sum((model_function(x_data, alpha_step) - y_data)**2) - np.sum((predictions - y_data)**2)) / 1e-5

        # Actualizar los parámetros usando el gradiente y la tasa de aprendizaje
        alpha -= learning_rate * grad

        # Calcular el error cuadrático medio
        mse = np.mean(error**2)
        if mse < tolerance:
            print(f"Convergió en {iteration} iteraciones.")
            break

    return alpha

# Cargar los datos desde el archivo usando np.loadtxt
file_path = '/content/datos_ejemplo_ajuste.dat'  # Cambia a la ruta correcta
data = np.loadtxt(file_path)

# Separar los datos en x e y
x_data, y_data = data[:, 0], data[:, 1]

# Parámetros iniciales para el descenso por gradiente
initial_parameters = [1, 1, 1]
learning_rate = 0.01

# Llamar a la función de descenso por gradiente
alpha_optimized = gradient_descent_least_squares(model_function, x_data, y_data, initial_parameters, learning_rate)

print("Parámetros optimizados:", alpha_optimized)


Parámetros optimizados: [-5.53645013  2.07226604 -7.45472829]


# 2) Comodidades


## 2.a) Vectorización (1pt)

Haga que el parámetro de condiciones iniciales esté vectorizado, es decir, que cuando sea una lista de parámetros, retorne una lista de resultados del método empezando desde cada especificación de parámetros iniciales.

No recomiendo usar `np.vectorize`, pues aunque es posible, es complicado usarlo en este caso.

## 2.b) Descenso estocástico (1pt)

agregue un parámetro a la función `train_test_divide=False`, que cuando sea un número diferente de cero, realice cada paso del descenso gradiente con una selección aleatoria de los datos `x` y `y`.

Por ejemplo, si `train_test_divide=0.35`, cada paso se debe realizar seleccionando al azar el 35% de las parejas (x,y).

In [22]:
import numpy as np

# Definir la función no lineal para el ajuste
def model_function(x, alpha):
    return alpha[0] * np.exp(-alpha[1] * x) * np.sin(alpha[2] * x)

# Función de descenso por gradiente
def gradient_descent_least_squares(model_function, x_data, y_data, initial_parameters, learning_rate, max_iterations=1000, tolerance=1e-6, train_test_divide=False):
    if isinstance(initial_parameters[0], list) or isinstance(initial_parameters[0], np.ndarray):
        # Si initial_parameters es una lista de listas, llamar recursivamente para cada conjunto de parámetros
        results = [gradient_descent_least_squares(model_function, x_data, y_data, params, learning_rate, max_iterations, tolerance, train_test_divide) for params in initial_parameters]
        return results

    alpha = np.array(initial_parameters, dtype=np.float64)
    N = len(y_data)

    for iteration in range(max_iterations):
        if train_test_divide:
            # Si train_test_divide es diferente de cero, seleccionar un porcentaje de los datos aleatoriamente
            indices = np.random.choice(N, int(N * train_test_divide), replace=False)
            x_batch = x_data[indices]
            y_batch = y_data[indices]
        else:
            x_batch = x_data
            y_batch = y_data

        # Calcular los valores predichos por el modelo
        predictions = model_function(x_batch, alpha)

        # Calcular el error entre las predicciones y los datos reales
        error = predictions - y_batch

        # Calcular el gradiente
        grad = np.zeros_like(alpha)
        for i in range(len(alpha)):
            # Derivada numérica del modelo con respecto a cada parámetro
            alpha_step = np.copy(alpha)
            alpha_step[i] += 1e-5  # Pequeño incremento para la derivada numérica
            grad[i] = (np.sum((model_function(x_batch, alpha_step) - y_batch)**2) - np.sum((predictions - y_batch)**2)) / 1e-5

        # Actualizar los parámetros usando el gradiente y la tasa de aprendizaje
        alpha -= learning_rate * grad

        # Calcular el error cuadrático medio
        mse = np.mean(error**2)
        if mse < tolerance:
            print(f"Convergió en {iteration} iteraciones.")
            break

    return alpha

# Cargar los datos desde el archivo usando np.loadtxt
file_path = '/content/datos_ejemplo_ajuste.dat'  # Cambia a la ruta correcta
data = np.loadtxt(file_path)

# Separar los datos en x e y
x_data, y_data = data[:, 0], data[:, 1]

# Parámetros iniciales para el descenso por gradiente (vectorizado)
initial_parameters = [[1, 1, 1], [0.5, 1.5, 0.8], [1.2, 0.8, 1.1]]
learning_rate = 0.01

# Llamar a la función de descenso por gradiente con descenso estocástico (ejemplo con 35% de datos seleccionados aleatoriamente)
train_test_divide = 0.35

alpha_optimized = gradient_descent_least_squares(model_function, x_data, y_data, initial_parameters, learning_rate, train_test_divide=train_test_divide)

print("Parámetros optimizados:", alpha_optimized)




Parámetros optimizados: [array([3.27645462, 0.97519928, 0.37335323]), array([5.53086858, 2.04372203, 7.4666428 ]), array([5.53585564, 2.10838071, 7.42451473])]


# 3) bonos

Encuentre una función que describa los datos `datos_09.npz`.

Si usa un polinomio o una serie de Fourier, encuentre el orden "óptimo", investigue qué significa eso.




